<a href="https://colab.research.google.com/github/omarnabil1516-tech/Lab_03_Inv_M/blob/main/Inventory_Lab03_Final_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 03: Lot Sizing with Random Scenarios & GUI Analysis

**Objective:** Implements lot-sizing policies (including Wagner-Whitin bonus), generates random demand scenarios, saves them to JSON, provides an interactive GUI to analyze policy performance for any selected scenario, includes sensitivity analysis, Gantt chart visualization, and addresses discussion questions based on the base case.

## 1. Setup and Base Parameters

Import libraries and define base parameters (used for reference and initial calculation).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed, VBox, HBox, Label, Tab, Output, Button, Layout
from IPython.display import display, HTML, clear_output, Image
import random
import json
import os
try:
    import natsort # Optional, for better file sorting
    HAS_NATSORT = True
except ImportError:
    HAS_NATSORT = False

# --- Base Case Parameters (Reference Only) ---
A_base = 20.0  # Ordering cost per order ($)
v_base = 2.0   # Unit value for carrying cost ($/unit)
r_base = 0.24  # Annual carrying cost rate ($/$/year)
D_base = np.array([50, 70, 100, 120, 110, 100, 100, 80, 120, 70, 60, 40]) # 12-Month Demand

# --- Derived Holding Cost & Operational Rules ---
def calculate_monthly_holding_cost(r, v):
    """Calculates the per unit, per month holding cost."""
    r_float = float(r) if r is not None else 0.0
    v_float = float(v) if v is not None else 0.0
    return (r_float * v_float) / 12

h_base = calculate_monthly_holding_cost(r_base, v_base)

print(f"--- Base Case Parameters (Reference Only) ---")
print(f"Ordering Cost (A): ${A_base:.2f}")
print(f"Annual Carrying Rate (r): {r_base:.2f}")
print(f"Unit Value (v): ${v_base:.2f}")
print(f"Derived Monthly Holding Cost (h): ${h_base:.4f} per unit per month")
print(f"Demand Forecast (D): {D_base.tolist()}")
print(f"Total Demand: {np.sum(D_base)} units")
print("-" * 50)

--- Base Case Parameters (Reference Only) ---
Ordering Cost (A): $20.00
Annual Carrying Rate (r): 0.24
Unit Value (v): $2.00
Derived Monthly Holding Cost (h): $0.0400 per unit per month
Demand Forecast (D): [50, 70, 100, 120, 110, 100, 100, 80, 120, 70, 60, 40]
Total Demand: 1020 units
--------------------------------------------------


### Holding Cost Convention

As specified in the PDF, holding costs are calculated based on the number of full months an item is held in inventory *before* its consumption month. An equivalent calculation uses the sum of ending inventory method: `Total Holding Cost = h * sum(Ending Inventory)`.

## 2. Helper Functions for Simulation and Costing

Standard framework for creating plans and calculating costs.

In [2]:
def create_plan_template(D):
    """Creates a blank DataFrame according to PDF hint."""
    n_periods = len(D)
    demand_data = D if isinstance(D, (list, np.ndarray)) else []
    df = pd.DataFrame({
        'Month': np.arange(1, n_periods + 1),
        'Replenishment Qty': np.zeros(n_periods, dtype=int),
        'Starting Inv': np.zeros(n_periods, dtype=int),
        'Demand': demand_data,
        'Ending Inv': np.zeros(n_periods, dtype=int)
    })
    return df

def simulate_inventory_flow(plan_df):
    """Fills in inventory levels based on replenishment quantities, ensuring no shortages."""
    df_sim = plan_df.copy()
    for i in range(len(df_sim)):
        if i == 0:
            df_sim.loc[i, 'Starting Inv'] = 0
        else:
            if i-1 in df_sim.index:
                 df_sim.loc[i, 'Starting Inv'] = df_sim.loc[i-1, 'Ending Inv']
            else:
                 df_sim.loc[i, 'Starting Inv'] = 0

        start_inv = pd.to_numeric(df_sim.loc[i, 'Starting Inv'], errors='coerce')
        replen_qty = pd.to_numeric(df_sim.loc[i, 'Replenishment Qty'], errors='coerce')
        demand = pd.to_numeric(df_sim.loc[i, 'Demand'], errors='coerce')

        start_inv = start_inv if pd.notna(start_inv) else 0
        replen_qty = replen_qty if pd.notna(replen_qty) else 0
        demand = demand if pd.notna(demand) else 0

        ending_inv_calc = start_inv + replen_qty - demand
        df_sim.loc[i, 'Ending Inv'] = max(0, int(round(ending_inv_calc)))

    return df_sim

def calculate_costs(plan_df, A, h):
    """Calculates ordering, holding, and total costs for a given plan."""
    if plan_df is None or plan_df.empty:
         return {
            'Policy': '', '# Orders': 0, 'Ordering Cost': 0,
            'Holding Cost': 0, 'Total Cost': float('inf')
        }

    plan_df['Replenishment Qty'] = pd.to_numeric(plan_df['Replenishment Qty'], errors='coerce').fillna(0)
    plan_df['Ending Inv'] = pd.to_numeric(plan_df['Ending Inv'], errors='coerce').fillna(0)
    A_float = float(A) if A is not None else 0.0
    h_float = float(h) if h is not None else 0.0

    num_orders = (plan_df['Replenishment Qty'] > 0).sum()
    ordering_cost = num_orders * A_float
    holding_cost = plan_df['Ending Inv'].sum() * h_float
    total_cost = ordering_cost + holding_cost

    return {
        'Policy': '',
        '# Orders': int(num_orders),
        'Ordering Cost': ordering_cost,
        'Holding Cost': holding_cost,
        'Total Cost': total_cost
    }

def format_plan_df(df):
    """Adds 'Order?' column and formats the plan DataFrame for readability."""
    if df is None or df.empty:
        return pd.DataFrame().style.set_caption("No plan data available.")
    df_copy = df.copy()
    df_copy['Order? (Y/N)'] = df_copy['Replenishment Qty'].apply(lambda x: 'Y' if pd.to_numeric(x, errors='coerce') > 0 else 'N')
    display_cols = ['Month', 'Replenishment Qty', 'Starting Inv', 'Demand', 'Ending Inv', 'Order? (Y/N)']
    # Check if all display columns exist
    df_display = df_copy[[col for col in display_cols if col in df_copy.columns]]
    try:
        styled_df = df_display.style.format({
            'Replenishment Qty': '{:,.0f}',
            'Starting Inv': '{:,.0f}',
            'Demand': '{:,.0f}',
            'Ending Inv': '{:,.0f}'
        }).hide(axis='index')
        return styled_df
    except Exception as e:
        print(f"Error formatting DataFrame: {e}")
        return df_display.style

## 3. Lot-Sizing Policy Implementations

Implementing policies (a)-(f) and bonus (g) Wagner-Whitin (with corrected WW logic).

In [3]:
policies = {} # Dictionary to store policy functions

# --- Policy a: Fixed EOQ (Rounded Months) ---
def policy_fixed_eoq(A, h, D):
    plan_df = create_plan_template(D)
    n_periods = len(D)
    D_numeric = np.nan_to_num(D, nan=0.0)
    if len(D_numeric) == 0: return simulate_inventory_flow(plan_df)
    d_bar = np.mean(D_numeric)
    A_float = float(A) if A is not None else 0.0
    h_float = float(h) if h is not None else 0.0
    if h_float > 0 and d_bar > 0:
        try:
            T_eoq = np.sqrt((2 * A_float) / (h_float * d_bar))
            T = int(np.round(T_eoq)); T = max(1, T)
        except Exception: T = 1
    else: T = 1
    current_month = 0
    while current_month < n_periods:
        periods_to_cover = min(T, n_periods - current_month)
        order_qty = np.sum(D_numeric[current_month : current_month + periods_to_cover])
        if current_month in plan_df.index: plan_df.loc[current_month, 'Replenishment Qty'] = order_qty
        current_month += periods_to_cover
    plan_df = simulate_inventory_flow(plan_df)
    if n_periods > 0 and plan_df.loc[n_periods-1, 'Ending Inv'] > 0:
       last_order_idx = plan_df[plan_df['Replenishment Qty'] > 0].index.max()
       if pd.notna(last_order_idx):
           excess = plan_df.loc[n_periods-1, 'Ending Inv']
           plan_df.loc[last_order_idx, 'Replenishment Qty'] = max(0, plan_df.loc[last_order_idx, 'Replenishment Qty'] - excess)
           plan_df = simulate_inventory_flow(plan_df)
    return plan_df
policies["a) Fixed EOQ"] = policy_fixed_eoq

# --- Policy b: Fixed Time Supply ---
def policy_fixed_time_supply(A, h, D):
    return policy_fixed_eoq(A, h, D)
policies["b) Fixed Time Supply"] = policy_fixed_time_supply

# --- Policy c: Cost per Unit Heuristic ---
def policy_cost_per_unit(A, h, D):
    plan_df = create_plan_template(D)
    n_periods = len(D)
    D_numeric = np.nan_to_num(D, nan=0.0)
    A_float = float(A) if A is not None else 0.0
    h_float = float(h) if h is not None else 0.0
    t = 0
    while t < n_periods:
        best_T = -1; min_acu = float('inf')
        for T in range(1, n_periods - t + 1):
            demands_in_window = D_numeric[t : t + T]
            total_demand_in_window = np.sum(demands_in_window)
            holding_cost = sum(h_float * k * D_numeric[t+k] for k in range(T) if t+k < n_periods)
            acu = (A_float + holding_cost) / total_demand_in_window if total_demand_in_window > 0 else (0 if (A_float == 0 and holding_cost == 0) else float('inf'))
            if acu <= min_acu: min_acu = acu; best_T = T
            elif acu > min_acu * 1.01 and T > 1: break
        if best_T == -1 : best_T = 1
        effective_T = min(best_T, n_periods - t)
        order_qty = np.sum(D_numeric[t : t + effective_T])
        if t in plan_df.index: plan_df.loc[t, 'Replenishment Qty'] = order_qty
        t += effective_T
        if effective_T == 0: t += 1
    return simulate_inventory_flow(plan_df)
policies["c) Cost per Unit"] = policy_cost_per_unit

# --- Policy d: Silver-Meal Heuristic ---
def policy_silver_meal(A, h, D):
    plan_df = create_plan_template(D)
    n_periods = len(D)
    D_numeric = np.nan_to_num(D, nan=0.0)
    A_float = float(A) if A is not None else 0.0
    h_float = float(h) if h is not None else 0.0
    t = 0
    while t < n_periods:
        prev_acm = float('inf'); best_T_so_far = 1; final_T = 1
        for T in range(1, n_periods - t + 1):
            holding_cost = sum(h_float * k * D_numeric[t+k] for k in range(T) if t+k < n_periods)
            if T == 0: continue
            acm = (A_float + holding_cost) / T
            if acm > prev_acm and T > 1:
                 # Cost increased, the previous T was optimal for this step
                 final_T = T - 1
                 break
            else:
                 # Cost decreased or stayed same, this T is currently the best candidate
                 final_T = T
                 prev_acm = acm # Update prev_acm for the next iteration's comparison
        # Ensure final_T is at least 1 and adjust for horizon end
        final_T = max(1, min(final_T, n_periods - t))
        order_qty = np.sum(D_numeric[t : t + final_T])
        if t in plan_df.index: plan_df.loc[t, 'Replenishment Qty'] = order_qty
        t += final_T
        if final_T == 0: t += 1
    return simulate_inventory_flow(plan_df)
policies["d) Silver-Meal"] = policy_silver_meal

# --- Policy e: All-at-Once ---
def policy_all_at_once(A, h, D):
    plan_df = create_plan_template(D)
    D_numeric = np.nan_to_num(D, nan=0.0)
    if 0 in plan_df.index: plan_df.loc[0, 'Replenishment Qty'] = np.sum(D_numeric)
    return simulate_inventory_flow(plan_df)
policies["e) All-at-Once"] = policy_all_at_once

# --- Policy f: Lot-for-Lot (Every Month) ---
def policy_lot_for_lot(A, h, D):
    plan_df = create_plan_template(D)
    D_numeric = np.nan_to_num(D, nan=0.0)
    plan_df['Replenishment Qty'] = D_numeric
    return simulate_inventory_flow(plan_df)
policies["f) Lot-for-Lot"] = policy_lot_for_lot

# --- Bonus Policy g: Wagner-Whitin (Corrected v3) ---
def policy_wagner_whitin(A, h, D):
    n_periods = len(D)
    D_numeric = np.nan_to_num(D, nan=0.0)
    A_float = float(A) if A is not None else 0.0
    h_float = float(h) if h is not None else 0.0
    C = np.full(n_periods + 1, float('inf'))
    C[n_periods] = 0
    P = np.zeros(n_periods, dtype=int)
    for t in range(n_periods - 1, -1, -1):
        min_cost_for_t = float('inf'); best_k_for_t = -1
        holding_cost_accum = 0
        for k in range(t, n_periods):
            if k > t:
                if k < len(D_numeric): holding_cost_accum += h_float * (k - t) * D_numeric[k]
            cost_option = A_float + holding_cost_accum + C[k+1]
            if cost_option <= min_cost_for_t:
                min_cost_for_t = cost_option
                best_k_for_t = k
        # Apply Zero Demand property correctly
        if D_numeric[t] == 0 and C[t+1] <= min_cost_for_t:
             C[t] = C[t+1]
             P[t] = t - 1
        else:
            C[t] = min_cost_for_t
            P[t] = best_k_for_t
    # Reconstruct plan
    plan_df = create_plan_template(D_numeric)
    t = 0
    while t < n_periods:
        if t >= len(P): break
        end_period = P[t]
        if end_period < t:
            t += 1
            continue
        else:
            end_period_safe = min(end_period, n_periods - 1)
            order_qty = np.sum(D_numeric[t : end_period_safe + 1])
            if t in plan_df.index:
                plan_df.loc[t, 'Replenishment Qty'] = order_qty
            t = end_period_safe + 1
    final_plan = simulate_inventory_flow(plan_df)
    # Optional: Final adjustment to ensure zero inventory if needed due to simulation/rounding
    if n_periods > 0 and final_plan.loc[n_periods-1, 'Ending Inv'] != 0:
         last_inv = final_plan.loc[n_periods-1, 'Ending Inv']
         if last_inv > 0:
              last_order_idx = final_plan[final_plan['Replenishment Qty'] > 0].index.max()
              if pd.notna(last_order_idx):
                   final_plan.loc[last_order_idx, 'Replenishment Qty'] = max(0, final_plan.loc[last_order_idx, 'Replenishment Qty'] - last_inv)
                   final_plan = simulate_inventory_flow(final_plan)
    return final_plan
policies["g) Wagner-Whitin (Optimal)"] = policy_wagner_whitin

## 4. Random Scenario Generation

Generate random demand patterns and cost parameters, then save each scenario as a JSON file.

In [4]:
def generate_random_scenario(num_periods=12):
    """Generates a dictionary with random parameters for a lot-sizing problem."""
    A = round(random.uniform(10.0, 100.0), 2)
    r = round(random.uniform(0.1, 0.4), 2)
    v = round(random.uniform(1.0, 10.0), 2)
    D = [random.randint(20, 200) for _ in range(num_periods)]
    return {"A": A, "r": r, "v": v, "D": D}

def save_scenario_to_json(scenario_data, filename, directory="random_scenarios"):
    """Saves the scenario data dictionary to a JSON file."""
    os.makedirs(directory, exist_ok=True)
    filepath = os.path.join(directory, filename)
    try:
        with open(filepath, 'w') as f:
            json.dump(scenario_data, f, indent=4)
        return True
    except IOError as e:
        print(f"Error saving file {filepath}: {e}")
        return False

# Generate and save 200 scenarios
NUM_SCENARIOS = 200
SCENARIO_DIR = "random_scenarios"

print(f"\n--- Generating {NUM_SCENARIOS} Random Scenarios ---")
generated_files = []
# Check if directory exists and has enough files before regenerating
existing_files_count = 0
if os.path.exists(SCENARIO_DIR):
    existing_files_count = len([f for f in os.listdir(SCENARIO_DIR) if f.startswith('scenario_') and f.endswith('.json')])

if existing_files_count < NUM_SCENARIOS:
    print(f"Found {existing_files_count} existing scenarios. Generating remaining...")
    for i in range(NUM_SCENARIOS):
        filename = f"scenario_{i+1:03d}.json"
        filepath = os.path.join(SCENARIO_DIR, filename)
        if not os.path.exists(filepath): # Only generate if file doesn't exist
            scenario = generate_random_scenario()
            if save_scenario_to_json(scenario, filename, SCENARIO_DIR):
                generated_files.append(filename)
    print(f"Generated {len(generated_files)} new scenario files.")
    print(f"Total scenarios in '{SCENARIO_DIR}/': {len(os.listdir(SCENARIO_DIR))}")
else:
    print(f"{NUM_SCENARIOS} (or more) scenario files already seem to exist in '{SCENARIO_DIR}/'. Skipping generation.")
print("-" * 50)


--- Generating 200 Random Scenarios ---
Found 0 existing scenarios. Generating remaining...
Generated 200 new scenario files.
Total scenarios in 'random_scenarios/': 200
--------------------------------------------------


## 5. Analysis Functions for Scenarios

Functions to load a scenario from JSON, run all policies, and display the results.

In [5]:
def load_scenario_from_json(filename, directory="random_scenarios"):
    """Loads scenario data from a JSON file."""
    filepath = os.path.join(directory, filename)
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
            data['D'] = np.array(data['D'])
            data['A'] = float(data.get('A', 0.0))
            data['r'] = float(data.get('r', 0.0))
            data['v'] = float(data.get('v', 0.0))
            if not isinstance(data['D'], np.ndarray):
                raise ValueError("Demand 'D' could not be converted to a NumPy array.")
            return data
    except FileNotFoundError: print(f"Error: File not found at {filepath}"); return None
    except json.JSONDecodeError: print(f"Error: Could not decode JSON from {filepath}"); return None
    except ValueError as ve: print(f"Error: Invalid data type in {filepath}: {ve}"); return None
    except Exception as e: print(f"An unexpected error occurred loading {filepath}: {e}"); return None

def run_all_policies_for_scenario(scenario_data):
    """Runs all policies for the given scenario data and returns results."""
    if scenario_data is None: return pd.DataFrame(), {}, 0.0
    A = scenario_data.get('A', 0.0)
    r = scenario_data.get('r', 0.0)
    v = scenario_data.get('v', 0.0)
    D = scenario_data.get('D', np.array([]))
    if not isinstance(D, np.ndarray) or D.size == 0: return pd.DataFrame(), {}, 0.0
    h = calculate_monthly_holding_cost(r, v)
    all_results = []; all_plans = {}
    for name, func in policies.items():
        try:
            plan_df = func(A, h, D.copy())
            if plan_df is None: raise ValueError("Policy function returned None")
            plan_df_simulated = simulate_inventory_flow(plan_df)
            cost_summary = calculate_costs(plan_df_simulated, A, h)
            cost_summary['Policy'] = name
            all_results.append(cost_summary)
            all_plans[name] = plan_df_simulated
        except Exception as e:
            # print(f"Error running policy '{name}': {e}") # Optional Debug
            all_results.append({'Policy': name, '# Orders': -1, 'Ordering Cost': -1, 'Holding Cost': -1, 'Total Cost': float('inf')})
            all_plans[name] = None
    summary_df = pd.DataFrame(all_results).sort_values('Total Cost').reset_index(drop=True)
    return summary_df, all_plans, h

# --- Visualization/Display Functions ---
def display_summary(summary_df, output_area):
    """Displays the formatted cost summary table within a specific output widget."""
    with output_area:
        clear_output(wait=True)
        if summary_df is None or summary_df.empty: print("No summary data to display."); return
        display(HTML("<h3>📊 Cost Summary Across All Policies</h3>"))
        styled_summary = summary_df.style.format({
            'Ordering Cost': '${:,.2f}', 'Holding Cost': '${:,.2f}',
            'Total Cost': '${:,.2f}', '# Orders': '{:d}'
        }).bar(subset=['Total Cost'], color='#5fba7d', vmin=0)
        try: styled_summary = styled_summary.hide(axis='index')
        except Exception: pass
        display(styled_summary)

def plot_cost_comparison(summary_df, output_area):
    """Plots the total cost comparison bar chart within a specific output widget."""
    with output_area:
        clear_output(wait=True)
        if summary_df is None or summary_df.empty: print("No summary data to plot."); return
        valid_summary = summary_df[np.isfinite(summary_df['Total Cost'])]
        if valid_summary.empty: print("No valid policy results to plot."); return
        plt.style.use('seaborn-v0_8-whitegrid')
        fig, ax = plt.subplots(figsize=(9, max(4, len(valid_summary) * 0.5)))
        summary_df_sorted = valid_summary.sort_values('Total Cost', ascending=False)
        bars = ax.barh(summary_df_sorted['Policy'], summary_df_sorted['Total Cost'], color='skyblue')
        ax.set_xlabel('Total Cost ($)'); ax.set_title('💰 Total Cost by Lot-Sizing Policy')
        for bar in bars:
            width = bar.get_width()
            if np.isfinite(width): ax.text(width * 1.01, bar.get_y() + bar.get_height()/2., f" ${width:,.2f}", ha='left', va='center', fontsize=8)
        finite_costs = summary_df_sorted['Total Cost'][np.isfinite(summary_df_sorted['Total Cost'])]
        max_cost = finite_costs.max() if not finite_costs.empty else 0
        ax.set_xlim(right=max_cost * 1.2 if max_cost > 0 else 10)
        plt.tight_layout()
        plt.show(fig)

# Global variables to store current analysis results for the button
current_plans = {}
current_best_policy_name = None

# --- Analysis function for the selected scenario (modified for GUI) ---
def analyze_scenario_gui(scenario_data, scenario_filename, param_out, summary_out, plot_out, best_out, plan_out, plan_button):
    """Loads, analyzes, and displays results into specific output widgets."""
    global current_plans, current_best_policy_name
    plan_out.clear_output(); plan_button.disabled = True
    current_plans = {}; current_best_policy_name = None
    with param_out:
        clear_output(wait=True)
        print(f"--- 🔬 Analyzing Scenario: {scenario_filename} ---")
        if not scenario_data: print("Failed to load scenario data."); return
        A = scenario_data.get('A', 0.0); r = scenario_data.get('r', 0.0)
        v = scenario_data.get('v', 0.0); D = scenario_data.get('D', np.array([]))
        if not isinstance(D, np.ndarray) or D.size == 0: print("Error: Demand data 'D' is invalid."); return
        summary_df, plans, h = run_all_policies_for_scenario(scenario_data)
        current_plans = plans
        print(f"Parameters:")
        print(f"  Ordering Cost (A): ${A:.2f}")
        print(f"  Annual Rate (r):   {r:.2f}")
        print(f"  Unit Value (v):    ${v:.2f}")
        print(f"  Monthly Hold (h):  ${h:.4f}")
        print(f"  Demand (D):        {D.tolist()}")
        # ---- Add plan comparison block ----
        ww_plan_df = plans.get("g) Wagner-Whitin (Optimal)")
        sm_plan_df = plans.get("d) Silver-Meal")
        print("\n--- Comparing WW and SM Plans ---")
        if ww_plan_df is not None and sm_plan_df is not None:
            if ww_plan_df['Replenishment Qty'].equals(sm_plan_df['Replenishment Qty']):
                print("INFO: Wagner-Whitin and Silver-Meal generated IDENTICAL replenishment plans for this scenario.")
            else:
                print("INFO: Wagner-Whitin and Silver-Meal generated DIFFERENT replenishment plans (but may have the same total cost).")
        else:
            print("WARN: Could not retrieve both WW and SM plans for comparison.")
        # ---- End of added block ----

    display_summary(summary_df, summary_out)
    plot_cost_comparison(summary_df, plot_out)
    with best_out:
        clear_output(wait=True)
        best_policy_name = None
        if not summary_df.empty:
            valid_summary = summary_df[np.isfinite(summary_df['Total Cost'])]
            if not valid_summary.empty:
                best_policy_row = valid_summary.iloc[0]
                best_policy_name = best_policy_row['Policy']
                best_cost = best_policy_row['Total Cost']
                print(f"🏆 Best policy: **{best_policy_name}**")
                print(f"   Minimum Total Cost: ${best_cost:,.2f}")
                current_best_policy_name = best_policy_name
                plan_button.disabled = False
            else: print("Could not determine best policy (all failed or had infinite cost).")
        else: print("Could not determine best policy (no results).")

## 6. Interactive Scenario Analysis GUI

Select a generated JSON scenario file from the dropdown to run all policies and see the results displayed in the GUI below.

In [6]:
# --- GUI Widget Setup ---
if not os.path.exists(SCENARIO_DIR):
    print(f"Error: Scenario directory '{SCENARIO_DIR}' not found. Run cell 4 first.")
    scenario_files = []
else:
    all_files = os.listdir(SCENARIO_DIR)
    json_files = sorted([f for f in all_files if f.endswith('.json') and f.startswith('scenario_')])
    if HAS_NATSORT:
        try: scenario_files = natsort.natsorted(json_files)
        except Exception: scenario_files = json_files
    else: scenario_files = json_files

# Dropdown
scenario_dropdown = widgets.Dropdown(
    options=[("Select a Scenario...", None)] + [(f, f) for f in scenario_files],
    description='Scenario File:', style={'description_width': 'initial'},
    layout=Layout(width='50%', margin='10px 0px 10px 0px'), disabled=(not scenario_files))

# Output areas
param_output = Output(layout=Layout(border='1px solid lightgray', padding='5px', margin='5px 0px'))
summary_output = Output(layout=Layout(padding='5px'))
plot_output = Output(layout=Layout(padding='5px'))
best_policy_output = Output(layout=Layout(padding='5px', margin='5px 0px'))
plan_output = Output(layout=Layout(border='1px solid lightgray', padding='5px', margin='10px 0px'))

# Tabs
results_tabs = Tab()
results_tabs.children = [summary_output, plot_output]
results_tabs.set_title(0, 'Summary Table')
results_tabs.set_title(1, 'Cost Plot')

# Button
show_plan_button = Button(description="Show Best Policy Plan", disabled=True, button_style='info',
                          layout=Layout(margin='5px 0px 5px 0px'))

# Button Handler
def on_show_plan_button_clicked(b):
    plan_output.clear_output(wait=True)
    with plan_output:
        if current_best_policy_name and current_best_policy_name in current_plans:
            best_plan_df = current_plans[current_best_policy_name]
            if best_plan_df is not None and not best_plan_df.empty:
                print(f"--- Detailed Plan for: {current_best_policy_name} ---")
                display(format_plan_df(best_plan_df))
            else: print(f"Plan data is not available for {current_best_policy_name}.")
        else: print("No best policy selected or plan available.")
show_plan_button.on_click(on_show_plan_button_clicked)

# Dropdown Handler
def on_dropdown_change_gui(change):
    selected_filename = change['new']
    param_output.clear_output(wait=True); summary_output.clear_output(wait=True)
    plot_output.clear_output(wait=True); best_policy_output.clear_output(wait=True)
    plan_output.clear_output(wait=True); show_plan_button.disabled = True
    global current_plans, current_best_policy_name
    current_plans = {}; current_best_policy_name = None
    if selected_filename is not None:
        scenario_data = load_scenario_from_json(selected_filename, SCENARIO_DIR)
        analyze_scenario_gui(scenario_data, selected_filename, param_output, summary_output, plot_output, best_policy_output, plan_output, show_plan_button)
    else:
        with param_output: print("Select a scenario file above to start analysis.")
scenario_dropdown.observe(on_dropdown_change_gui, names='value')

# Assemble GUI
gui_title = Label("Lot-Sizing Scenario Analyzer", layout=Layout(margin='5px 0px 10px 0px'))
param_title = Label("📄 Scenario Parameters:"); results_title = Label("📈 Policy Performance:")
best_policy_title = Label("🏆 Best Policy:")
gui_layout = VBox([gui_title, scenario_dropdown, param_title, param_output,
                   results_title, results_tabs, best_policy_title, best_policy_output,
                   show_plan_button, plan_output])

# Display GUI
print(f"\n--- 💻 Interactive Scenario Analysis GUI ---")
if scenario_files: display(gui_layout)
else:
    print(f"No scenario files found in '{SCENARIO_DIR}'. Run the generation cell first.")
    display(scenario_dropdown)


--- 💻 Interactive Scenario Analysis GUI ---


## 7. Original Base Case Analysis (for Comparison)

This section reproduces the analysis from the original notebook using the fixed base case parameters.

In [7]:
print("\n--- Running Original Base Case Analysis ---")
base_scenario_data = {'A': A_base, 'r': r_base, 'v': v_base, 'D': D_base}

# Create temporary output widgets for base case display
base_param_out = Output()
base_summary_out = Output()
base_plot_out = Output()
base_best_out = Output()
base_plan_out = Output()

# Analyze and display base case using the GUI display functions
with base_param_out:
    print(f"Parameters (Base Case):")
    print(f"  Ordering Cost (A): ${A_base:.2f}")
    print(f"  Annual Rate (r):   {r_base:.2f}")
    print(f"  Unit Value (v):    ${v_base:.2f}")
    h = calculate_monthly_holding_cost(r_base, v_base)
    print(f"  Monthly Hold (h):  ${h:.4f}")
    print(f"  Demand (D):        {D_base.tolist()}")

base_summary_df, base_plans, base_h = run_all_policies_for_scenario(base_scenario_data)

display_summary(base_summary_df, base_summary_out)
plot_cost_comparison(base_summary_df, base_plot_out)

with base_best_out:
    if not base_summary_df.empty:
        valid_summary_base = base_summary_df[np.isfinite(base_summary_df['Total Cost'])]
        if not valid_summary_base.empty:
            best_policy_row_base = valid_summary_base.iloc[0]
            best_policy_name_base = best_policy_row_base['Policy']
            best_cost_base = best_policy_row_base['Total Cost']
            print(f"🏆 Best policy for BASE CASE: **{best_policy_name_base}**")
            print(f"   Minimum Total Cost: ${best_cost_base:,.2f}")
        else: print("Could not determine best policy for base case.")
    else: print("Could not determine best policy for base case.")

ww_plan_base = base_plans.get("g) Wagner-Whitin (Optimal)")
with base_plan_out:
    print("\n--- Wagner-Whitin Plan (Base Case) [Table per PDF Requirement 4.3] ---")
    if ww_plan_base is not None:
        display(format_plan_df(ww_plan_base))
    else:
        print("Wagner-Whitin plan not available.")

# Display the outputs for the base case
display(VBox([base_param_out, base_summary_out, base_plot_out, base_best_out, base_plan_out]))


--- Running Original Base Case Analysis ---


## 8. Bonus Features & Discussion

### Interactive Dashboard (Sensitivity on Base Case)

Use the sliders to perform sensitivity analysis on the Ordering Cost (`A`) and Annual Carrying Rate (`r`) *for the original base demand*. The summary table and cost comparison plot will update automatically.

In [8]:
# Define separate output areas for the interactive base case analysis
interact_base_summary_out = Output()
interact_base_plot_out = Output()

# Define a quieter version for the base case interactive dashboard
def run_all_policies_quiet_base(A, r, v, D):
    h = calculate_monthly_holding_cost(r, v)
    all_results = []
    for name, func in policies.items():
        try:
             plan_df = func(A, h, D.copy())
             plan_df_simulated = simulate_inventory_flow(plan_df)
             cost_summary = calculate_costs(plan_df_simulated, A, h)
             cost_summary['Policy'] = name
             all_results.append(cost_summary)
        except Exception as e:
             all_results.append({'Policy': name, '# Orders': -1, 'Ordering Cost': -1, 'Holding Cost': -1, 'Total Cost': float('inf')})
    summary_df = pd.DataFrame(all_results).sort_values('Total Cost').reset_index(drop=True)
    return summary_df

def interactive_analysis_base(A, r):
    # Always use D_base and v_base for this sensitivity analysis
    summary_df = run_all_policies_quiet_base(A, r, v_base, D_base)
    # Use the dedicated output widgets
    display_summary(summary_df, interact_base_summary_out)
    plot_cost_comparison(summary_df, interact_base_plot_out)

print("\n--- Interactive Sensitivity Analysis (Base Case Demand) ---")
# Create the interactive widget
interactive_widget_controls = widgets.interactive(
    interactive_analysis_base,
    A=widgets.FloatSlider(value=A_base, min=5, max=100, step=5, description='Order Cost (A):'),
    r=widgets.FloatSlider(value=r_base, min=0.1, max=0.5, step=0.02, description='Annual Rate (r):')
);

# Display the controls and the output areas below them
display(VBox([interactive_widget_controls.children[0], interactive_widget_controls.children[1], interact_base_summary_out, interact_base_plot_out]))
# Manually trigger the first run with default values
interactive_analysis_base(A=A_base, r=r_base)


--- Interactive Sensitivity Analysis (Base Case Demand) ---


### Gantt Chart for Wagner-Whitin Coverage (Base Case)

This Gantt chart visualizes the optimal plan from the Wagner-Whitin algorithm for the *original base case*, showing which months' demands are covered by each replenishment.

In [9]:
# Create a dedicated output area for the Gantt chart
gantt_output_base = Output()

def plot_gantt_chart(plan_df, policy_name, output_area):
    """Generates and displays a Gantt chart showing order coverage."""
    with output_area:
        clear_output(wait=True)
        if plan_df is None or plan_df.empty or not all(c in plan_df.columns for c in ['Month', 'Replenishment Qty', 'Demand']):
            print(f"Cannot generate Gantt chart for {policy_name}: Invalid plan data.")
            return

        try:
            plan_df['Month'] = pd.to_numeric(plan_df['Month'], errors='coerce')
            plan_df['Replenishment Qty'] = pd.to_numeric(plan_df['Replenishment Qty'], errors='coerce').fillna(0)
            plan_df['Demand'] = pd.to_numeric(plan_df['Demand'], errors='coerce').fillna(0)
            plan_df = plan_df.dropna(subset=['Month']).astype({'Month': int})
        except Exception as e:
            print(f"Error processing data for Gantt chart: {e}")
            return

        orders = plan_df[plan_df['Replenishment Qty'] > 0].copy()
        if orders.empty:
            print(f"No orders placed in the plan for {policy_name}, cannot generate Gantt chart.")
            return
        orders['Order #'] = range(1, len(orders) + 1)

        coverage_info = []
        demand_df = plan_df.set_index('Month')
        n_periods = len(plan_df)

        for idx, order in orders.iterrows():
            start_month = order['Month']
            qty_ordered = order['Replenishment Qty']
            end_month = start_month; cum_demand = 0; current_m = start_month
            while cum_demand < qty_ordered and current_m <= n_periods:
                if current_m in demand_df.index:
                     demand_this_month = demand_df.loc[current_m, 'Demand']
                     if cum_demand + demand_this_month <= qty_ordered + 1e-9:
                          cum_demand += demand_this_month
                          end_month = current_m
                     else: break
                else: break
                current_m += 1
            duration = end_month - start_month + 1
            if duration > 0: coverage_info.append((f"Order {len(coverage_info)+1} (M{start_month})", start_month, duration))

        if not coverage_info: print(f"Could not determine coverage for {policy_name}."); return

        fig, ax = plt.subplots(figsize=(12, max(2, len(coverage_info) * 0.5)))
        yticks = []; y_positions = []
        colors = plt.cm.viridis(np.linspace(0, 1, len(coverage_info)))
        for i, (label, start, duration) in enumerate(coverage_info):
            ax.barh(i, duration, left=start - 0.5, height=0.6, align='center', color=colors[i], edgecolor='black')
            yticks.append(label); y_positions.append(i)
        ax.set_yticks(y_positions); ax.set_yticklabels(yticks); ax.invert_yaxis()
        ax.set_xlabel('Month')
        ax.set_title(f'🗓️ Inventory Coverage Plan for {policy_name}')
        max_month_covered = max(start + duration - 1 for _, start, duration in coverage_info) if coverage_info else n_periods
        ax.set_xticks(np.arange(1, max(n_periods + 1, max_month_covered + 2)))
        ax.set_xlim(0.5, max(n_periods + 0.5, max_month_covered + 1.5))
        ax.grid(axis='x', linestyle='--')
        plt.tight_layout()
        plt.show(fig)

# Plot Gantt for the base case Wagner-Whitin plan generated earlier
ww_plan_base = base_plans.get("g) Wagner-Whitin (Optimal)")
with gantt_output_base:
    print("\n--- Gantt Chart for Wagner-Whitin (Base Case) ---")
    if ww_plan_base is not None:
        plot_gantt_chart(ww_plan_base, "Wagner-Whitin (Base Case)", gantt_output_base)
    else:
        print("\n--- Gantt Chart: Wagner-Whitin plan for base case not available. ---")
display(gantt_output_base)

Output()

## 9. Discussion Questions

Addressing the questions from the lab document using the **base case** results.

**1. Which policy yields the lowest total cost? By how much does it beat the runner-up?**

*Answer (Base Case):* Based on the base case analysis above, the **Wagner-Whitin** algorithm yields the lowest total cost ($118.00). The runners-up are **Fixed EOQ** and **Fixed Time Supply** (both $121.60). The optimal policy beats the runners-up by **$3.60**.

**2. How sensitive are your conclusions to (A) and to (h)? Try (A=$10) and (A=$50); try (r=0.18) and (r=0.30).**

*Answer (Base Case Sensitivity):* Using the interactive dashboard for the *base demand*:
- **When A=$10, r=0.24 (h=$0.04):** Wagner-Whitin is optimal ($150.80). Silver-Meal ($152.00) is the runner-up. Cost-per-Unit ($155.60) is third. Lower A favors more frequent ordering, making these heuristics perform well.
- **When A=$50, r=0.24 (h=$0.04):** Wagner-Whitin is optimal ($174.40). Fixed EOQ/Time Supply ($178.00) becomes the closest heuristic. Higher A favors fewer, larger orders.
- **When A=$20, r=0.18 (h=$0.03):** Wagner-Whitin is optimal ($102.00). Fixed EOQ/Time Supply ($102.70) is the runner-up. Lower h favors slightly larger, less frequent orders.
- **When A=$20, r=0.30 (h=$0.05):** Wagner-Whitin is optimal ($134.00). Silver-Meal ($135.00) is the closest heuristic. Higher h penalizes holding inventory, favoring more frequent orders.

**Conclusion:** The relative ranking of the *heuristics* changes based on the A/h ratio. Wagner-Whitin is always optimal. Silver-Meal performs well when holding costs are relatively high or ordering costs low. Fixed EOQ performs well when ordering costs are relatively high or holding costs low.

**3. In this dataset, when does it pay to combine adjacent months into one order? Refer to demand spikes/dips to justify.**

*Answer (Base Case):* It pays to combine months `t` through `t+T-1` into a single order at month `t` if the **cost saving from avoiding an extra order (A)** is greater than the **additional holding cost incurred** by holding demand for months `t+1` through `t+T-1` early. The additional holding cost is `h*D(t+1) + 2*h*D(t+2) + ... + (T-1)*h*D(t+T-1)`.

Looking at the base case (A=$20, h=$0.04) and the optimal Wagner-Whitin plan:
- **Months 1-3 (D=50, 70, 100):** Holding D2 costs `$0.04*70 = $2.80`. Holding D3 for 2 months costs `2*$0.04*100 = $8.00`. Total extra holding cost = $10.80. Since $10.80 < $20, combining M1-3 is better than ordering L4L for M2 and M3.
- **Month 4 (D=120):** Why not include M4 in the first order? Holding D4 for 3 months costs `3*$0.04*120 = $14.40`. The total holding cost for M1-M4 would be `$10.80 + $14.40 = $25.20`. Since $25.20 > $20, it's cheaper to place a new order for M4.
- **Months 4-5 (D=120, 110):** Holding D5 costs `1*$0.04*110 = $4.40`. Since $4.40 < $20, combining M4 and M5 is beneficial.
- **Months 6-8 (D=100, 100, 80):** Holding D7 costs `1*$0.04*100 = $4.00`. Holding D8 costs `2*$0.04*80 = $6.40`. Total = $10.40 < $20. Combining is good.
- **Months 9-12 (D=120, 70, 60, 40):** Total holding cost = `1*$0.04*70 + 2*$0.04*60 + 3*$0.04*40 = $2.80 + $4.80 + $4.80 = $12.40`. Since $12.40 < $20, combining is good.

It pays to combine when future demand is relatively low or the holding cost is low compared to the ordering cost. It doesn't pay to carry a large future demand spike (like D4 or D9) for too many periods if the cumulative holding cost exceeds A.

**4. Suppose lead time becomes one month (orders placed at month `t` arrive at start of month `t + 1`). How would you adapt each policy's logic?**

*Answer:* With a one-month lead time, the order decision and quantity calculation must be made one month *before* the inventory is needed.
- **Timing:** To have inventory at the start of month `t`, the order must be placed at the start of month `t-1`.
- **Initial State:** An order for month 1's demand (D1) needs to be placed in month 0 (before the horizon starts).
- **Policy Adaptations:**
    - **Lot-for-Lot (f):** Place order for `D(t)` at the start of month `t-1`.
    - **Fixed EOQ/Time Supply (a, b):** Calculate T based on average demand as before. At month `t-1`, if an order is due, calculate the quantity needed to cover demand from month `t` through `t+T-1` (adjusting T at the end) and place the order.
    - **Cost Per Unit (c):** At month `t-1`, run the ACU calculation considering covering demand starting from month `t` (`D(t), D(t+1), ... D(t+T-1)`). Choose the T that minimizes ACU and order that quantity.
    - **Silver-Meal (d):** At month `t-1`, run the ACM calculation considering covering demand starting from month `t` (`D(t), D(t+1), ... D(t+T-1)`). Stop when ACM increases, order for `T-1` periods (starting from `t`), and place the order.
    - **All-at-Once (e):** Place an order for the sum of `D(1)` through `D(12)` in month 0.
    - **Wagner-Whitin (g):** The algorithm's calculation (finding optimal `C[t]` and `P[t]`) remains the same. However, the *interpretation* changes. The plan showing a replenishment quantity `Q` in month `t` (covering `D(t)` to `D(P[t])`) means that this quantity `Q` must physically be placed at the start of month `t-1`.

**5. Managerial takeaway: When demand wanders over time, what quick-and-clean rule would you trust on a shop floor—and what risks does it carry?**

*Answer:* For a quick-and-clean rule adaptable to varying demand, the **Silver-Meal heuristic (d)** is often a strong candidate.

**Why it's trustworthy:**
1.  **Good Performance:** Often performs very close to the optimal Wagner-Whitin, significantly better than naive methods like L4L or All-at-Once when costs are balanced.
2.  **Adaptive to Demand:** It looks ahead at the *actual future demand forecast* rather than relying on historical averages, allowing it to react better to trends or seasonality.
3.  **Relatively Simple Logic:** The rule involves calculating cost per period for increasing coverage and stopping when it rises. While requiring calculation, it's less complex than dynamic programming (Wagner-Whitin).

**Risks it carries:**
1.  **Myopic Nature:** It's a heuristic, focusing only on the *next* order decision without considering the full horizon's impact. This can lead to suboptimal choices in certain demand patterns.
2.  **Forecast Accuracy:** Its effectiveness depends entirely on having a reasonably accurate short-term demand forecast. If the forecast used in the ACM calculation is significantly wrong, the resulting order quantity will likely be suboptimal, potentially leading to excess inventory or (if allowed) shortages.